In [1]:
!nvidia-smi

Wed Jan  6 11:24:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 455.45.01    CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3070    Off  | 00000000:09:00.0  On |                  N/A |
|  0%   40C    P8    12W / 220W |    253MiB /  7979MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tensorflow "tf-models-official"

In [3]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer
import pandas as pd

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [6]:
df=pd.read_csv("train.csv")
df.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [7]:
df.shape

(1306122, 3)

In [8]:
df.target.value_counts()/df.shape[0]

0    0.93813
1    0.06187
Name: target, dtype: float64

In [9]:
X = df.question_text
y = df.target

In [10]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [11]:
AUTOTUNE

-1

In [12]:
def create_dataset(X, Y, epochs, batch_size):
# Using the tf.data.Dataset.from_tensor_slices() method we are able to get the slices of list or array
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

In [13]:
full_dataset = create_dataset(X, y, epochs=1, batch_size=128)
# train_ds = train_ds_raw.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
df.shape

(1306122, 3)

In [15]:
DATASET_SIZE = 1306122

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [16]:
for x in train_ds.take(1):
    print(x)

(<tf.Tensor: shape=(128,), dtype=string, numpy=
array([b'How did Quebec nationalists see their province as a nation in the 1960s?',
       b'Do you have an adopted dog, how would you encourage people to adopt and not shop?',
       b'Why does velocity affect time? Does velocity affect space geometry?',
       b'How did Otto von Guericke used the Magdeburg hemispheres?',
       b'Can I convert montra helicon D to a mountain bike by just changing the tyres?',
       b'Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?',
       b'Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?',
       b'Is it crazy if I wash or wipe my groceries off? Germs are everywhere.',
       b'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?',
       b'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disreg

In [17]:
class_names = {0 : "sincire", 1: "insincire"}

In [18]:
for text_batch, label_batch in train_ds.take(1):
    for i in range(3):
        print(f'question: {text_batch.numpy()[i]}')
        label = label_batch.numpy()[i]
        print(f'Label : {label} ({class_names[label]})')

question: b'How did Quebec nationalists see their province as a nation in the 1960s?'
Label : 0 (sincire)
question: b'Do you have an adopted dog, how would you encourage people to adopt and not shop?'
Label : 0 (sincire)
question: b'Why does velocity affect time? Does velocity affect space geometry?'
Label : 0 (sincire)


In [19]:
#@title Choose a BERT model to fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'  #@param ["bert_en_uncased_L-12_H-768_A-12", "bert_en_cased_L-12_H-768_A-12", "bert_multi_cased_L-12_H-768_A-12", "small_bert/bert_en_uncased_L-2_H-128_A-2", "small_bert/bert_en_uncased_L-2_H-256_A-4", "small_bert/bert_en_uncased_L-2_H-512_A-8", "small_bert/bert_en_uncased_L-2_H-768_A-12", "small_bert/bert_en_uncased_L-4_H-128_A-2", "small_bert/bert_en_uncased_L-4_H-256_A-4", "small_bert/bert_en_uncased_L-4_H-512_A-8", "small_bert/bert_en_uncased_L-4_H-768_A-12", "small_bert/bert_en_uncased_L-6_H-128_A-2", "small_bert/bert_en_uncased_L-6_H-256_A-4", "small_bert/bert_en_uncased_L-6_H-512_A-8", "small_bert/bert_en_uncased_L-6_H-768_A-12", "small_bert/bert_en_uncased_L-8_H-128_A-2", "small_bert/bert_en_uncased_L-8_H-256_A-4", "small_bert/bert_en_uncased_L-8_H-512_A-8", "small_bert/bert_en_uncased_L-8_H-768_A-12", "small_bert/bert_en_uncased_L-10_H-128_A-2", "small_bert/bert_en_uncased_L-10_H-256_A-4", "small_bert/bert_en_uncased_L-10_H-512_A-8", "small_bert/bert_en_uncased_L-10_H-768_A-12", "small_bert/bert_en_uncased_L-12_H-128_A-2", "small_bert/bert_en_uncased_L-12_H-256_A-4", "small_bert/bert_en_uncased_L-12_H-512_A-8", "small_bert/bert_en_uncased_L-12_H-768_A-12", "albert_en_base", "electra_small", "electra_base", "experts_pubmed", "experts_wiki_books", "talking-heads_base"]

map_name_to_handle = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
}

map_model_to_preprocess = {
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    }

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1


In [20]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

In [21]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [22]:
classifier_model = build_classifier_model()

In [23]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [24]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [25]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [26]:
classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
preprocessing (KerasLayer)      {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
BERT_encoder (KerasLayer)       {'pooled_output': (N 28763649    preprocessing[0][0]              
                                                                 preprocessing[0][1]              
                                                                 preprocessing[0][2]              
______________________________________________________________________________________________

In [27]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "bert_classifier", monitor='val_loss', verbose=0, save_best_only=False,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

In [28]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_ds, 
                               validation_data=val_ds,
                               epochs=epochs,callbacks = [checkpoint])

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/5
10204/10204 [==============================] - 5210s 510ms/step - loss: 0.1757 - binary_accuracy: 0.9318


Epoch 2/5
10204/10204 [==============================] - 5212s 511ms/step - loss: 0.0949 - binary_accuracy: 0.9605


Epoch 3/5
10204/10204 [==============================] - 5244s 514ms/step - loss: 0.0822 - binary_accuracy: 0.9657


Epoch 4/5
10204/10204 [==============================] - 5253s 515ms/step - loss: 0.0717 - binary_accuracy: 0.9705


Epoch 5/5
10204/10204 [==============================] - 5253s 515ms/step - loss: 0.0640 - binary_accuracy: 0.9739


In [29]:
classifier_model.save("model_5eps_9739binAcc")